In [1]:
import numpy as np
np.random.seed(0)

# ¡Escala tus *pipelines* de procesado de datos mixtos sin salir de scikit-learn!

## Pedro Morales <pedro.morales@ebury.com>
## Enrique Colin <enrique.colin@ebury.com>

![ebury](img/ebury_logo.jpg)
![pycones](img/pycones_logo.png)

### PyConES 2018 @ Málaga, 2018-10-06

# ¿Por qué scikit-learn?

![sklearn](img/sklearn.png)

* Extensa colección de algoritmos y herramientas auxiliares para transformar *datasets* y construir modelos
* API muy expresiva y notación altamente estandarizada
* Permite fácil integración de funcionalidades personalizadas
* Proyecto muy activo en constante mejora
* **Python**!


# `Pipeline` - ¿Para qué sirve?

![mariopipe](img/mariopipe.jpg)

* En su *camino* hacia el estimador final, someteremos a los datos a transformaciones intermedias como p.ej.:
  - Tratamiento de `NaN`s: `sklearn.impute.SimpleImputer`
  - Encoding a tipos numéricos: `sklearn.preprocessing.OneHotEncoder`
  - Reducción de dimensiones
      + *Feature selection*: `sklearn.feature_selection.SelectFromModel`
      + *Feature extraction*: `sklearn.decomposition.PCA`
  
  
> `sklearn.pipeline.Pipeline` nos ayuda a **encapsular** cadenas de transformaciones **secuenciales** sobre nuestros datos

## Ejemplo: *Titanic dataset*

In [2]:
import pandas as pd

data = pd.read_csv((
    'https://raw.githubusercontent.com/amueller/'
    'scipy-2017-sklearn/091d371/notebooks/datasets/titanic3.csv')
)

data.head(2)

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"


Supongamos que queremos predecir la probabilidad de supervivencia `survived` con la edad `age` y precio de billete `fare`.

In [3]:
columns = ['fare', 'age']
target = 'survived'

X = data[columns].values
y = data[target].values

Componemos una cadena de transformación donde:
+ imputamos `NaN`'s, 
+ escalamos las variables, 
+ y finalmente predecimos con una regresión logística.

In [4]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

clf = Pipeline(steps=[('imputer', SimpleImputer()),
                      ('scaler', StandardScaler()),
                      ('classifier', LogisticRegression(solver='lbfgs'))])

Esta encapsulación nos permite operar fácilmente con el modelo completo, tratándolo como un único objeto. 

In [5]:
from sklearn.model_selection import cross_val_score

cv_scores = cross_val_score(clf, X, y, cv=5)
print('CV accuracy: %0.3f' % cv_scores.mean())
print('"0" class prior: %0.3f' % (1 - y.mean()))

CV accuracy: 0.633
"0" class prior: 0.618


## pero además...
La correcta encapsulación de operaciones es **esencial** en la etapa de **validación** (leakage...)

![leakage](img/leakage.jpg)

## `Pipeline` - ¿cubre toda la casuística de *arquitecturas* de modelos?

## No
* Incorporando **diferentes tipos de datos** al modelo, el preprocesado y extracción de *features* consistirá en **varias cadenas** de operaciones **paralelas**, no en una única rama secuencial


![model_example](img/model_example.png)

>## ¿Y qué hay de `sklearn.pipeline.FeatureUnion`?
>
>* No nos da toda la flexibilidad que necesitamos: aplica diferentes cadenas de operaciones (paralelas) al **mismo** conjunto de datos
>* No permite **selectividad** sobre columnas.

# Antes de scikit-learn 0.20

## Alternativa 1: *Do it yourself*

* Desarrollo *propio*, con el apoyo de las clases presentes `sklearn.base` (herencia de `TransformerMixin`,  `BaseEstimator`...) 

![esfuerzo](img/sisifo.jpg)

## Alternativa 2: [sklearn-pandas](https://github.com/scikit-learn-contrib/sklearn-pandas)

>This module provides a bridge between Scikit-Learn's machine learning methods and pandas-style Data Frames.
>
>In particular, it provides:
>
>1. **A way to map DataFrame columns to transformations, which are later recombined into features.**
>
> ...

* Funcionalidad estrella: `DataFrameMapper`
* Constituye la semilla de `sklearn.compose.ColumnTransformer`.

# sklearn 0.20 y la herramienta prometida: `ColumnTransformer`

## Volvamos a nuestro ejemplo...

In [6]:
data.head(2)

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"


Construyamos un modelo mas complejo, con las siguientes variables:
+ Variables numéricas:
    - `age`: float.
    - `fare`: float.
+ Variables categóricas, discretas:
    - `embarked`: categorías codificadas como strings {`'C'`, `'S'`, `'Q'`}.
    - `sex`: categorías codificadas como strings {`'female'`, `'male'`}.
    - `pclass`: enteros ordinales {`1`, `2`, `3`}.

Las transformaciones adecuadas para cada variable depende (entre otras cosas) de su tipo. P.ej.:
- Variables numéricas:
    1. Imputación de `NaN`s con la mediana
    2. Escalado estándar
- Variables categóricas:
    1. Imputación de `NaN`s con una nueva categoría `missing`
    2. *One hot encoding* (dando solo ceros a categorías no vistas en entrenamiento)

- Variables numéricas:
    1. Imputación de `NaN`s con la mediana
    2. Escalado estándar

In [7]:
numeric_features = ['age', 'fare']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

- Variables categóricas:
    1. Imputación de `NaN`s con una nueva categoría `missing`
    2. *One hot encoding* (dando solo ceros a categorías no vistas en entrenamiento)

In [8]:
from sklearn.preprocessing import OneHotEncoder

categorical_features = ['embarked', 'sex', 'pclass']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

### Encapsulamos las cadenas de preproceso de variables categóricas y numéricas en un unico objeto:

In [9]:
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

Finalmente, empaquetamos el módulo de preproceso con el estimador:

In [10]:
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', LogisticRegression(solver='lbfgs'))])

Y ya podemos aplicar operaciones selectivas sobre nuestro `DataFrame` de pandas:

In [11]:
X = data.drop('survived', axis=1)
y = data['survived']

cv_scores = cross_val_score(clf, X, y, cv=5)
print('CV accuracy: %0.3f' % cv_scores.mean())
print('"0" class prior: %0.3f' % (1 - y.mean()))

CV accuracy: 0.646
"0" class prior: 0.618


# ¿Preguntas?

# ¡Muchas gracias!